In [30]:
 !pip install nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
from keras.models import Sequential
from keras.layers import Input, Embedding,TimeDistributed, LSTM, Dropout, Bidirectional, Dense 
import random
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model, to_categorical
import nltk
from tqdm import tqdm
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


True

In [39]:
def preprosses(src,top_decoupled):
    word_tag=[]
    for src,trg in  zip(src,top_decoupled):
        out=extract_bio_tags(src,trg)
        word_tag.append(out)
    return word_tag




In [44]:
file_path = "/kaggle/input/dataset/src.txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    src = np.array([line.strip() for line in file])

file_path = "/kaggle/input/dataset/top_decoupled.txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    top_decoupled = np.array([line.strip() for line in file])

In [45]:
def extract_bio_tags(src,top_decoupled):
    # Parse the TOP-DECOUPLED structure
    matches = re.split(r"(?=\(PIZZAORDER|\(DRINKORDER)", top_decoupled)
    start=matches[0].split()
    start=start[1:]
    matches=matches[1:]
    start_index=len(start)
    
    
    
    
    # Map words to BIO tags
    bio_tag_map = {}
    src_words = src.split()
    
    for i,match in enumerate(matches):
        
        match=re.sub(r'\(|\)','',match)
        match=match.strip()
        words= match.split()  # Split multi-word values
        tag=words[0]
        words=words[1:]
        for i, word in enumerate(words):
            if i == 1:
                if word in src_words[start_index:]:
                    bio_tag_map[src_words.index(word,start_index)]= f"B-{tag}"  # Begin tag
                    start_index+=1
            else:
                if word in src_words[start_index:]:
                    bio_tag_map[src_words.index(word,start_index)]= f"I-{tag}"  # Inside tag
                    start_index+=1            
  
    ind=-1
    keys=list(bio_tag_map.keys())
    if random.random() < 0.5:
         ind=random.randint(0, len(keys)-1)
    word_tag_pairs=[(src_words[(i)],bio_tag_map.get(i)) for i in bio_tag_map.keys()]
 
    if ind!=-1:
        word_tag_pairs[ind]=("<UNK>",bio_tag_map.get(keys[ind]))
    
    return word_tag_pairs
 





In [46]:
sentences =preprosses(src,top_decoupled)


In [48]:


maxlen = max([len(s) for s in sentences])
words = [w[0] for sentence in sentences for w in sentence]
words = list(set(words))
words.append("<pad>")
n_words = len(words)
print('Number of unique words:', n_words)


Number of unique words: 323


In [49]:


tags = [w[1] for sentence in sentences for w in sentence]
tags=list(set(tags))
tags.append("O")
print(tags)
n_tags = len(tags)
print('Number of unique Tags:', n_tags)

['B-PIZZAORDER', 'I-PIZZAORDER', 'I-DRINKORDER', 'B-DRINKORDER', 'O']
Number of unique Tags: 5


In [50]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
print(tag2idx)

{'B-PIZZAORDER': 0, 'I-PIZZAORDER': 1, 'I-DRINKORDER': 2, 'B-DRINKORDER': 3, 'O': 4}


In [79]:
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = sequence.pad_sequences(maxlen=maxlen+40, sequences=X, padding="post",value=n_words - 1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = sequence.pad_sequences(maxlen=maxlen+40, sequences=y, padding="post", value=tag2idx['O'])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])
print('X shape', X.shape, 'y shape', y.shape)

X shape (15704, 58) y shape (15704, 58, 5)


In [86]:
class config():
    VOCAB = n_words
    MAX_LEN = maxlen+40
    N_OUPUT = n_tags
    
    
    EMBEDDING_VECTOR_LENGTH = 256
    N_LSTM_CELLS = 150
    RECURRENT_DROPOUT = 0.4
    
    OUTPUT_ACTIVATION = 'softmax'
    
    LOSS = 'categorical_crossentropy'
    OPTIMIZER = 'adam'
    METRICS = ['accuracy']
    
    MAX_EPOCHS = 5
    

In [87]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
filepath = "model.keras"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
rlp = ReduceLROnPlateau(monitor='loss', patience=3, verbose=1)

model = Sequential()
model.add(
    Embedding(
        input_dim=config.VOCAB, output_dim=config.EMBEDDING_VECTOR_LENGTH, input_length=config.MAX_LEN
    )
)
model.add(
    Bidirectional(
        LSTM(config.N_LSTM_CELLS, return_sequences=True, recurrent_dropout=config.RECURRENT_DROPOUT)
    )
)
model.add(
    TimeDistributed(
        Dense(config.N_OUPUT, activation=config.OUTPUT_ACTIVATION)
    )
)
model.compile(loss=config.LOSS, optimizer=config.OPTIMIZER, metrics=config.METRICS)



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [88]:
history = model.fit(x=X, y=y, validation_split=0.1,
    callbacks=[es, ckpt, rlp], epochs=config.MAX_EPOCHS
)

Epoch 1/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.9665 - loss: 0.1105
Epoch 1: loss improved from inf to 0.03556, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 59s 123ms/step - accuracy: 0.9666 - loss: 0.1104 - val_accuracy: 0.9990 - val_loss: 0.0036 - learning_rate: 0.0010
Epoch 2/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.9988 - loss: 0.0035
Epoch 2: loss improved from 0.03556 to 0.00303, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 53s 121ms/step - accuracy: 0.9988 - loss: 0.0035 - val_accuracy: 0.9992 - val_loss: 0.0027 - learning_rate: 0.0010
Epoch 3/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.9993 - loss: 0.0022
Epoch 3: loss improved from 0.00303 to 0.00198, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 54s 123ms/step - accuracy: 0.9993 - loss: 0.0022 - val_accuracy: 0.9994 - val_loss: 0.0020 - learning_rate: 0.0010
Epoch 4/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.9995 - loss: 0.0

In [89]:
dev_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_dev.json", lines=True)
src_dev=dev_data["dev.SRC"].to_numpy()
top_dev=dev_data["dev.TOP"].to_numpy()


In [90]:
def evaluate(test_src,test_top):
    true_len=0

    true=0
    for src,top in zip(test_src,test_top):
        top=re.sub(r'(?<=\))(?:[^\(\)]+?)(?=\()', '', top)
        
        src_words_orginal = src.split() 
        if(len(src_words_orginal)>config.MAX_LEN):
            continue
        true_len+=1
        for word in src_words_orginal:
            if word not in words:
                str=f"(?<=\\s){word}(?=\\s)"
                src=re.sub(str,"<UNK>",src)
        src_words=src.split()
        
        




        matches = re.split(r"(?=\(PIZZAORDER|\(DRINKORDER)", top)
        start=matches[0].split()
        start=start[1:]
        matches=matches[1:]
        start_index=len(start)
        bio_tag_map={}
        for match in matches:
            match=re.sub(r'\(|\)','',match)
            match=match.strip()
            match_words= match.split()  # Split multi-word values
            tag=match_words[0]
            match_words=match_words[1:]
            for i, word in enumerate(match_words):
                if i == 1:
                    if word in src_words_orginal[start_index:]:
                        bio_tag_map[src_words_orginal.index(word,start_index)]= f"B-{tag}"  # Begin tag
                        start_index+=1
                else:
                    if word in src_words_orginal[start_index:]:
                        bio_tag_map[src_words_orginal.index(word,start_index)]= f"I-{tag}"  # Inside tag
                        start_index+=1
        keys=list(bio_tag_map.keys())
        word_tag_pairs=[(src_words[(i)],bio_tag_map.get(i)) for i in bio_tag_map.keys()]
        test_src_gold=[wor[0] for wor in word_tag_pairs ]
        word_tag_pairs=[wor[1] for wor in word_tag_pairs ]
        word_tag_pairs = word_tag_pairs + ["O"] * (config.MAX_LEN - len(word_tag_pairs))

        
        

        src_indecies = [word2idx.get(w,word2idx.get("UNK", -1)) for w in test_src_gold]
        pad_token = word2idx.get("<pad>")  
        src_indecies = src_indecies + [pad_token] * (config.MAX_LEN - len(test_src_gold))
        src_indecies = np.expand_dims(src_indecies, axis=0)  # Shape: (1, 25)
        p = model.predict(src_indecies)  
        p = np.argmax(p, axis=-1)  # Get the predicted tag indices
        
        idx2tag = {v: k for k, v in tag2idx.items()}  
        # Iterate through predictions (p) and print the tags
        predicted=[]
        
        for i,w in enumerate(p[0]):  # p is likely a batch, so take the first sequence
            tag_begin=idx2tag[w]
            # if tag_begin[0:2]=="I-" and i>0 and tag_begin[2:-1]!=predicted[-1][2:-1] :
            #     tag_begin=re.sub("I-","B-",tag_begin)
            predicted.append(tag_begin) 
        
        acc=[a == b for a, b in zip(word_tag_pairs, predicted)]
        if(all(acc)==True):
            true+=1
     
    print("accuracy",true/true_len*100,"%")

In [92]:
evaluate(src_dev,top_dev)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━

In [58]:
def predict(sentence):
    
  
        src_words_orginal = sentence.split() 
        if(len(src_words_orginal)>config.MAX_LEN):
            return 
        for word in src_words_orginal:
            if word not in words:
                str=f"(?<=\\s){word}(?=\\s)"
                sentence=re.sub(str,"<UNK>",sentence)
        src_words=sentence.split()
        src_indecies = [word2idx.get(w,word2idx.get("UNK", -1)) for w in src_words]
        pad_token = word2idx.get("<pad>")  
        src_indecies = src_indecies + [pad_token] * (config.MAX_LEN - len(src_words))
        src_indecies = np.expand_dims(src_indecies, axis=0)  # Shape: (1, 25)
        p = model.predict(src_indecies)  
        p = np.argmax(p, axis=-1)  # Get the predicted tag indices
        
        idx2tag = {v: k for k, v in tag2idx.items()}  
        # Iterate through predictions (`p`) and print the tags
        predicted=[]
        
        for i,w in enumerate(p[0]):  # `p` is likely a batch, so take the first sequence
            predicted.append(idx2tag[w]) 
      
       
       
        
            

predict("I will take 2 pizza large with foul and salami")    
                
       
    

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
I <UNK> <UNK> 2 <UNK> large with <UNK> <UNK> salami ['B-PIZZAORDER', 'B-PIZZAORDER', 'B-PIZZAORDER', 'B-PIZZAORDER', 'B-PIZZAORDER', 'B-PIZZAORDER', 'B-PIZZAORDER', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [98]:
import json

with open("word2idx_model2.json", "w") as file:
    json.dump(word2idx,file)
with open("tag2idx_model2.json", "w") as file:
    json.dump(tag2idx,file)